In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
users = glueContext.create_dynamic_frame.from_catalog(database="default", table_name="userdatamlcertfinal_user_data_kinesis_firehouse2020", encoding="utf-8")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
print("Count: ", users.count())
users.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count:  5148
root
|-- FIRST: string
|-- LAST: string
|-- AGE: int
|-- GENDER: string
|-- LATITUDE: string
|-- LONGITUDE: string
|-- partition_0: string
|-- partition_1: string
|-- partition_2: string

In [95]:
print(users)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
users_df = users.toDF()
users_df.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+---+------+--------+---------+-----------+-----------+-----------+
|  FIRST|   LAST|AGE|GENDER|LATITUDE|LONGITUDE|partition_0|partition_1|partition_2|
+-------+-------+---+------+--------+---------+-----------+-----------+-----------+
|   النا|  یاسمی| 34|female| 33.6251| 177.8837|         11|         03|         16|
|Hüseyin| Pierre| 24|  male|-52.8681|-168.2833|         11|         03|         16|
|  Cesar|    Gil| 65|  male|-25.3485| -35.4932|         11|         03|         16|
|  Begüm|Tunçeri| 38|female| 74.5838| 128.9262|         11|         03|         16|
|Hüseyin| Pierre| 24|  male|-52.8681|-168.2833|         11|         03|         16|
|  Cesar|    Gil| 65|  male|-25.3485| -35.4932|         11|         03|         16|
|   النا|  یاسمی| 34|female| 33.6251| 177.8837|         11|         03|         16|
|  Begüm|Tunçeri| 38|female| 74.5838| 128.9262|         11|         03|         16|
|  Cesar|    Gil| 65|  male|-25.3485| -35.4932|         11|         03|     

In [37]:
num_males = users_df.where("GENDER = 'male'").count()
num_females = users_df.where("GENDER = 'female'").count()
num_total = users_df.count()

print("Percentage Males: ", str(round((num_males/num_total)*100, 2)), "%")
print("Percentage Females: ", str(round((num_females/num_total)*100, 2)), "%")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Percentage Males:  50.39 %
Percentage Females:  49.61 %

In [48]:
print("Top 5 Ages")
users_df.groupBy("AGE").count().orderBy("count", ascending=False).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 5 Ages
+---+-----+
|AGE|count|
+---+-----+
| 24| 1233|
| 34| 1230|
| 65| 1225|
| 38| 1206|
| 67|   12|
+---+-----+
only showing top 5 rows

In [69]:
print("Age bins by decades up to 100")

from pyspark.ml.feature import Bucketizer
bucketizer  =  Bucketizer().setInputCol("AGE").setOutputCol("AGE_BIN").setSplits((0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100 ))

binned_data = bucketizer.transform(users_df)

binned_data.groupBy("AGE_BIN").count().orderBy("count", ascending=False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Age bins by decades up to 100
+-------+-----+
|AGE_BIN|count|
+-------+-----+
|    3.0| 2469|
|    2.0| 1276|
|    6.0| 1272|
|    5.0|   51|
|    4.0|   50|
|    7.0|   30|
+-------+-----+

In [85]:
print("Transform Gender to Binary using String Indexer")

from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer().setInputCol("GENDER").setOutputCol("BINARY_GENDER")

indexed_gender_user_df = string_indexer.fit(users_df).transform(users_df)

indexed_gender_user_df = indexed_gender_user_df.drop("GENDER", "partition_0", "partition_1", "partition_2")
indexed_gender_user_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Transform Gender to Binary using String Indexer
+-------+-------+---+--------+---------+-------------+
|  FIRST|   LAST|AGE|LATITUDE|LONGITUDE|BINARY_GENDER|
+-------+-------+---+--------+---------+-------------+
|   النا|  یاسمی| 34| 33.6251| 177.8837|          1.0|
|Hüseyin| Pierre| 24|-52.8681|-168.2833|          0.0|
|  Cesar|    Gil| 65|-25.3485| -35.4932|          0.0|
|  Begüm|Tunçeri| 38| 74.5838| 128.9262|          1.0|
|Hüseyin| Pierre| 24|-52.8681|-168.2833|          0.0|
|  Cesar|    Gil| 65|-25.3485| -35.4932|          0.0|
|   النا|  یاسمی| 34| 33.6251| 177.8837|          1.0|
|  Begüm|Tunçeri| 38| 74.5838| 128.9262|          1.0|
|  Cesar|    Gil| 65|-25.3485| -35.4932|          0.0|
|Hüseyin| Pierre| 24|-52.8681|-168.2833|          0.0|
|   النا|  یاسمی| 34| 33.6251| 177.8837|          1.0|
|  Cesar|    Gil| 65|-25.3485| -35.4932|          0.0|
|  Begüm|Tunçeri| 38| 74.5838| 128.9262|          1.0|
|   النا|  یاسمی| 34| 33.6251| 177.8837|          1.0|
|Hüseyin| Pierre|

In [86]:
print("Writing Spark df to CSV in S3")

indexed_gender_user_df.write.csv("s3://preshen-sagemaker-sme-project/transformed-user-data-spark-csv", header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Writing Spark df to CSV in S3